In [305]:
import aoc
import logging
import numpy as np
import scipy
import matplotlib as mpl
import matplotlib.pyplot as plt
import itertools
import math
from typing import List
%reload_ext autoreload

day = 13
sample = False
sample_number = 1

In [306]:
input_list = aoc.split_contents(aoc.read_input(f'Input/day_{day:02}{"_sample"+str(sample_number) if sample else ""}.txt'))

# Part 1

In [307]:
pattern = list()
pattern_list = list()
# build a list of 2D arrays for each pattern
for i,row in enumerate(input_list):
    if len(row) == 0 and i != 0:
        pattern_list.append(np.array(pattern))
        pattern = []
    else:
        pattern.append([x for x in row])
pattern_list.append(np.array(pattern))

In [308]:
def pretty_print(array):
    for elem in array:
        print(''.join(elem))

In [309]:
def find_matching_columns(array):
    candidates = list()
    for col in range(len(array.T)-1):
        if np.array_equal(array[:,col], array[:,col+1]):
            candidates.append([col,col+1])
    return candidates

def find_matching_rows(array):
    candidates = list()
    for row in range(len(array)-1):
        if np.array_equal(array[row,:], array[row+1,:]):
            candidates.append([row,row+1])
    return candidates

def split_pattern_on_column(array, column_to_split_along):
    p_left = np.split(array, [column_to_split_along], 1)[0]
    p_right = np.split(array, [column_to_split_along], 1)[1]
    return np.fliplr(p_left), p_right

def split_pattern_on_row(array, row_to_split_along):
    p_up = np.split(array, [row_to_split_along], 0)[0]
    p_down = np.split(array, [row_to_split_along], 0)[1]
    return np.flipud(p_up), p_down

In [310]:
aoc.logger.setLevel(30)
summarized_value = 0
reflection_solutions = dict()  # used in Part 2
for i, p in enumerate(pattern_list):
    # We only need to consider the cases with adjacent rows or columns are equal to check for reflections.
    row_candidates = find_matching_rows(p)
    col_candidates = find_matching_columns(p)
    aoc.logger.info(f'{row_candidates=} {col_candidates=}')

    for col_candidate in col_candidates:
        array_l, array_r = split_pattern_on_column(p, col_candidate[1])
        narrowest_array = min(len(array_l.T), len(array_r.T))
        # check if the trimmed and reflected arrays are equal
        if np.array_equal(np.split(array_l,[narrowest_array],1)[0],
                          np.split(array_r,[narrowest_array],1)[0]):
            aoc.logger.info(f'Pattern: {i} vertical reflection found. {col_candidate[1]} columns to the left.')
            summarized_value += col_candidate[1]
            reflection_solutions[i] = ['V', col_candidate[1]]

    for row_candidate in row_candidates:
        array_u, array_d = split_pattern_on_row(p, row_candidate[1])
        shortest_array = min(len(array_u), len(array_d))
        if np.array_equal(np.split(array_u,[shortest_array],0)[0],
                          np.split(array_d,[shortest_array],0)[0]):
            aoc.logger.info(f'Pattern: {i} horizontal reflection found. {row_candidate[1]} rows above.')
            summarized_value += 100 * row_candidate[1]
            reflection_solutions[i] = ['H', row_candidate[1]]

In [311]:
summarized_value

27300

# Part 2

In [312]:
reflection_solutions

{0: ['H', 3],
 1: ['H', 3],
 2: ['H', 4],
 3: ['H', 4],
 4: ['V', 12],
 5: ['H', 1],
 6: ['V', 16],
 7: ['H', 7],
 8: ['H', 10],
 9: ['H', 12],
 10: ['H', 1],
 11: ['H', 1],
 12: ['H', 15],
 13: ['V', 8],
 14: ['H', 6],
 15: ['V', 12],
 16: ['H', 10],
 17: ['V', 14],
 18: ['V', 16],
 19: ['V', 13],
 20: ['V', 1],
 21: ['H', 3],
 22: ['V', 1],
 23: ['H', 16],
 24: ['H', 1],
 25: ['H', 4],
 26: ['V', 6],
 27: ['H', 7],
 28: ['V', 12],
 29: ['H', 9],
 30: ['V', 14],
 31: ['V', 12],
 32: ['V', 13],
 33: ['V', 8],
 34: ['H', 1],
 35: ['V', 2],
 36: ['V', 10],
 37: ['V', 1],
 38: ['V', 12],
 39: ['V', 1],
 40: ['V', 2],
 41: ['V', 12],
 42: ['H', 7],
 43: ['V', 3],
 44: ['V', 12],
 45: ['H', 8],
 46: ['H', 2],
 47: ['V', 14],
 48: ['H', 2],
 49: ['H', 9],
 50: ['H', 1],
 51: ['V', 15],
 52: ['H', 10],
 53: ['V', 12],
 54: ['H', 1],
 55: ['H', 6],
 56: ['H', 6],
 57: ['V', 11],
 58: ['H', 3],
 59: ['V', 6],
 60: ['V', 4],
 61: ['H', 1],
 62: ['V', 3],
 63: ['H', 4],
 64: ['H', 15],
 65: ['H',

In [313]:
aoc.logger.setLevel(30)
summarized_value = 0
for i, p2 in enumerate(pattern_list):
    reflection_found = False
    for x in range(len(p2.T)):
        for y in range(len(p2)):
            p = np.copy(p2)
            if p2[y][x] == '#':
                p[y][x] = '.'
            else:
                p[y][x] = '#'

            # We only need to consider the cases with adjacent rows or columns are equal to check for reflections.
            row_candidates = find_matching_rows(p)
            col_candidates = find_matching_columns(p)
            aoc.logger.info(f'{x=} {y=} {row_candidates=} {col_candidates=}')

            for col_candidate in col_candidates:
                array_l, array_r = split_pattern_on_column(p, col_candidate[1])
                narrowest_array = min(len(array_l.T), len(array_r.T))
                # check if the trimmed and reflected arrays are equal
                if np.array_equal(np.split(array_l,[narrowest_array],1)[0],
                                  np.split(array_r,[narrowest_array],1)[0]):
                    aoc.logger.info(f'Pattern: {i} vertical reflection found. {col_candidate[1]} columns to the left.')
                    if ((reflection_solutions[i][0] == 'V' and col_candidate[1] != reflection_solutions[i][1])
                            or reflection_solutions[i][0] == 'H'):
                        summarized_value += col_candidate[1]
                        reflection_found = True

            for row_candidate in row_candidates:
                array_u, array_d = split_pattern_on_row(p, row_candidate[1])
                shortest_array = min(len(array_u), len(array_d))
                if np.array_equal(np.split(array_u,[shortest_array],0)[0],
                                  np.split(array_d,[shortest_array],0)[0]):
                    aoc.logger.info(f'Pattern: {i} horizontal reflection found. {row_candidate[1]} rows above.')
                    if ((reflection_solutions[i][0] == 'H' and row_candidate[1] != reflection_solutions[i][1])
                            or reflection_solutions[i][0] == 'V'):
                        summarized_value += 100 * row_candidate[1]
                        reflection_found = True

            if reflection_found:
                break
        if reflection_found:
            break

In [314]:
summarized_value

29276